# Plotting quality control and baseline figures of the spatio-temporal network

**Author**: Emre Havazli, David Bekaert - Jet Propulsion Laboratory

This notebook provides an overview of the functionality included in the **`ariaPlot.py`** program. Specifically, we give examples on how the **`ariaPlot.py`** program can be used to generate both qualitiative and coverage plots  of ARIA Geocoded UNWrapped interferogram (GUNW) products over a user defined area of interest.

In this notebook, we will demonstrate how to extract:
- Qualitative Plots
  - Perpendicular baseline - Time plot
  - Coherence - Time plot
- Coverage plots
  - Lat extent
    

<div class="alert alert-danger">
<b>Potential Errors:</b> 
    
- GDAL uses "HDF5" driver instead of "netCDF/Network Common Data Format" on GUNW products. Verify GDAL version >= 3.
- ARIA-tools needs to be installed to run this notebook
</div>

## Overview of the ariaPlot.py program

We can print out help and usage of the tool using the command shown in the cell below. The tags which can be used and what they are used for is explanied in the output.

The **`ariaPlot.py`** program allows for easy generation of qualitatibe and spatial-temproal coverage plots over a user-defiend area of itnerest. Running **`ariaPlot.py`** with the **`-h`** option, will show the parameters options. 

Let us explore these options are:

In [1]:
!ariaPlot.py -h

usage: ariaPlot.py [-h] -f IMGFILE [-w WORKDIR] [-b BBOX] [-m MASK]
                   [-of OUTPUTFORMAT] [-croptounion] [-plottracks]
                   [-plotbperp] [-plotbperpcoh] [-plotcoh] [-makeavgoh]
                   [-plotall] [-verbose]

Function to generate various quality control and baseline figures of the
spatial-temporal network of products.

optional arguments:
  -h, --help            show this help message and exit
  -f IMGFILE, --file IMGFILE
                        ARIA file
  -w WORKDIR, --workdir WORKDIR
                        Specify directory to deposit all outputs. Default is
                        local directory where script is launched.
  -b BBOX, --bbox BBOX  Provide either valid shapefile or Lat/Lon Bounding
                        SNWE. -- Example : '19 20 -99.5 -98.5'
  -m MASK, --mask MASK  Mask file. This file needs to be GDAL compatible and
                        contains projection information its meta-data.
  -of OUTPUTFORMAT, --outputFormat OUTP

### 1. Product files to be used (-f option)

At minimum, users need to specify the GUNW files where they want to extract information from. This is controlled using the **`-f`** option. Multiple products can be specified by providing them as a comma separated string (e.g., **`-f`**` 'S1-GUNW-A-R-124-tops-20180502_20180408-043106-21658N_19755N-PP-0dd0-v2_0_1.nc,S1-GUNW-A-R-124-tops-20180502_20180408-043040-20161N_18088N-PP-6704-v2_0_1.nc'`), or using a wildcard (e.g., **`-f`**` 'S1*.nc'`).

### 4. Cropping and spatial sub setting (-b and -croptounion options)

The **`ariaPlot.py`** program will automatically handle cropping and stitching of GUNW products when needed. By default, the program will crop all interferograms to bounds determined by the common intersection (of all interferograms) and the user-defined bounding box option. All layers are cropped and/or stitched using GDAL (see the methods section for details on the implemented approach for each layer). Below we discuss which options are available for specifying an area-of-interest.

#### Interferogram intersection versus union (-croptounion option)

GUNW products are grouped in clusters that belong to the same interferometric pair. By default, the spatial **intersection** of the interferometic pairs is used to define the region of interest. This can be overwritten to be the union of all interferograms (regardless of alignment) by passing the **`--croptounion`** argument. A schematic example is shown in **Fig 2** for both scenario's.

<div class="alert alert-warning">
<b>Warning:</b> Users in general should avoid mixing products of adjacent satellite tracks (i.e., products made on the same contiguous pass are ok). Note that along the equator, the track number (ascending data on the ascending note) gets incremented while the data itself is still continuous.
</div>

<img src="support_docs/spatial_config_new.png" alt="spatial_configuration" width="700">
<blockquote> <center><b> Fig. 2 </b> Schematic of the spatiotemporal configuration for three interferograms. The left panel shows the intersection of the interferograms, which is the default behavior. The right panel shows the union of the interferograms, achieved by passing <b><code>--croptounion</code></b> . The blue dashed line demonstrates the behavior if the user had specified a bounding box (<b><code>-b</code></b>). Note that interferograms which do not cover the bounding box completely with the <b><code>--croptounion</code></b> or the <b><code>-b</code></b> option, will be patched with no-data values.</center></blockquote>

#### Bounding Box (-b option)

The user can specify a rectangular bounding box (South-North-West-East coordinates) as **`-b`**` 'S N W E'` or can provide a more complex area of interest using a shapefile or geoJSON as **`-b`**` path_to_file.shp`. A schematic example is shown in **Fig 3** for both scenario's.

<img src="./support_docs/crop.png" alt="cropping" width="700">
<blockquote><center><b> Fig. 3 </b> Schematic examples of the <b><code>-b</code></b> option available for cropping. The blue dashed line shows the user-specified area of interest. The left panel demonstrates the use when specifying SNWE coordinates, while the right panel shows when specifying a more complex polygon for the area of interest using a shapefile. </center></blockquote>

### 5. Workdirectory (-w option)

The output of the **`ariaPlot.py`** program is saved within the working directory (**`-w`**), which by default the current directory. Within the work directory the figure outputs are stored within the *figures* directory 

### 6.  Output format (-o option)

The **`ariaPlot.py`** program leverages GDAL for file reading and writing of outputs. Although most of the outputs are figures, users can specify any GDAL compatible format (e.g., ENVI, ISCE, GTiff; see GDAL for more information on supported format) for saving the output from the **`--makeavgoh`** option.

### 0.2. Input and output standards
The cell below is showing the usage of input and output tags. -f/--file tag is the only required tag. The command will create an empty folder with the name "outdir" in your working directory. You can change the name and location of the folder as needed.

In [3]:
!ariaPlot.py -f ./data/S1-GUNW-A-R-077-tops-20190222_20190210-231605-42666N_40796N-PP-d75b-v2_0_1.nc -w ./outdir

***Plotting Function:***
All 1 input files successfully passed!
Group spatiotemporally continuous IFGs.

All 1 input files successfully passed!


## 1. Qualitative Plots
### 1.1. Baseline Plots
We can analyze the quality of ARIA products by examining their baseline and coherence information. productPlot.py allows us to visualize this information from the product directly.

In [2]:
!ariaPlot.py -f ./data/S1-GUNW-A-R-077-tops-20190222_20190210-231605-42666N_40796N-PP-d75b-v2_0_1.nc -w ./outdir -plotbperp

***Plotting Function:***
All (1) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (1) interferograms are spatially continuous.
- Make baseline plot and histogram.


The output of the command above produces two figures in your workdir. The first figure is called **bperp_histogram.eps** and shows the distribution of perpendicular baselines with respect to number of interferograms. This figure allows us to visualize the distribtuion of perpendicular baselines in your data set, or a single product as in this case

<img src="support_docs/bperp_histogram.png" alt="Histogram" width="700">

The second figure we produced is perpendicular baseline vs time plot with the name **bperp_plot.eps**. This plot visualizes the interferometric pairs with respect to their perpendicular baselines.

<img src="support_docs/bperp_plot.png" alt="Baseline" width="1000">


We can use the same command on multiple ARIA products as seen below

In [4]:
!ariaPlot.py -f './data/*.nc' -w ./outdir -plotbperp

***Plotting Function:***
All (10) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (10) interferograms are spatially continuous.
- Make baseline plot and histogram.
Baseline plot warning!
Design matrix is rank deficient. Network is disconnected.
Using a fully connected network is recommended.


The resulting histogram of the randomly selected sample data set can be seen below.

<img src="support_docs/bperp_histogram_multi.png" alt="Histogram" width="700">

Perpnedicular baseline vs Time plot shows us the acquisition dates and perpendicular baseline relations of interferograms.
<img src="support_docs/bperp_plot_multi.png" alt="Baseline" width="1000" class="center">


However, regular perpendicular baseline vs time plot doesn't give us a lot of information about the quality of our interferograms. We can use coherence information along with perpendicular baselines and have a better understanding of the quality of ARIA products in our data set. The cell below is going to produce the perpendicular baseline vs time plot and use color code the lines which are representing interferograms with respect to their average coherence.

**Note:** We need to include -croptounion option because our data set has no common, overlapping, area.

In [6]:
!ariaPlot.py -f './data/*.nc' -w ./outdir -plotbperpcoh -croptounion

***Plotting Function:***
All 10 input files successfully passed!
Group spatiotemporally continuous IFGs.

All 10 input files successfully passed!
- Make baseline plot that is color-coded with respect to mean IFG coherence.
Baseline plot warning!
Design matrix is rank deficient. Network is disconnected.
Using a fully connected network is recommended.


<img src="support_docs/bperp_plot_multi_color.png" alt="Baseline" width="1000">

### 1.2. Coherence Plots

productPlot.py can be used to visualize coherence information of single or stacks of interferograms. The cell below produces plots such as number of interferograms vs average coherence and average coherence vs time.

In [7]:
!ariaPlot.py -f ./data/S1-GUNW-A-R-077-tops-20190222_20190210-231605-42666N_40796N-PP-d75b-v2_0_1.nc -w ./outdir -plotcoh

***Plotting Function:***
All 1 input files successfully passed!
Group spatiotemporally continuous IFGs.

All 1 input files successfully passed!
- Make average IFG coherence plot in time, and histogram of average IFG coherence.


<img src="support_docs/avgcoherence_histogram.png" alt="Histogram" width="700">

<img src="support_docs/avgcoherence_plot.png" alt="Avearge Coherence" width="1000">


The cell below shows how to use our tool with a multiple ARIA products. Again, we need to pass -croptounion tag since we don't have a common area between the products in our data set.

In [8]:
!ariaPlot.py -f './data/*.nc' -w ./outdir -plotcoh -croptounion

***Plotting Function:***
All 10 input files successfully passed!
Group spatiotemporally continuous IFGs.

All 10 input files successfully passed!
- Make average IFG coherence plot in time, and histogram of average IFG coherence.


<img src="support_docs/avgcoherence_histogram_multi.png" alt="Histogram" width="700">

<img src="support_docs/avgcoherence_plot_multi.png" alt="Avearge Coherence" width="1000">

## 2. Coverage Plots
We can produce coverage plots using -plottracks tag to visualize the latitude extents of each interferogram in our dataset.  

In [5]:
!ariaPlot.py -f 'data/*.nc' -w outdir -plottracks -croptounion

***Plotting Function:***
All (10) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (10) interferograms are spatially continuous.
- Make plot of track latitude extents vs bounding bbox/common track extent.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


<img src="support_docs/lat_extents.png" alt="Latitude Extents" width="1000">